In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict

data_path = "./data/toxic_spans"
filename = "toxic_span_text_pairs.csv"
splits = ["train", "eval", "test"]

dataset_dict = DatasetDict(
    {
        split: Dataset.from_pandas(
            pd.read_csv(
                data_path + "/" + filename.replace(".csv", "_" + split + ".csv")
            )
        )
        for split in splits
    }
)

In [3]:
import torch


def detox(tokenizer, model_buffered, batched_inputs):
    input_sequence_buffered = tokenizer(
        batched_inputs, padding="longest", pad_to_multiple_of=8, return_tensors="pt"
    ).to("cuda")
    with torch.no_grad():
        output_sequence = model_buffered.generate(
            input_ids=input_sequence_buffered["input_ids"],
            attention_mask=input_sequence_buffered["attention_mask"],
        )
    del input_sequence_buffered
    generated = [
        "<CSD>" if len(generated) == 0 else generated
        for generated in tokenizer.batch_decode(
            output_sequence, skip_special_tokens=True
        )
    ]
    return {"generated": generated}


In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
from datasets import Dataset

for model_name in ["t5-base", "t5-v1_1-base", "bart-base"]:
    model_name = f"ChrisZeng/{model_name}-detox"

    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_buffered = model.to("cuda")
    tokenizer.padding_side = "left"
    tokenizer.pad_token = tokenizer.eos_token

    dataset_dict = dataset_dict.map(
        lambda rec: detox(tokenizer, model_buffered, rec["original"]),
        keep_in_memory=True,
        batched=True,
        batch_size=64,
    ).rename_columns({"generated": f"generated_{model_name}"})

    del model_buffered

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

  0%|          | 0/136 [00:00<?, ?ba/s]

  0%|          | 0/49 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox'],
        num_rows: 8679
    })
    eval: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox'],
        num_rows: 3720
    })
})

In [5]:
import re


class SentenceDetoxer(object):
    def __init__(self, toxic_words):
        self.replacements = {toxic_word: "<CSD>" for toxic_word in toxic_words}
        self.pattern = "|".join(r"\b%s\b" % re.escape(s) for s in self.replacements)

    def __call__(self, sentence):
        return re.sub(
            self.pattern, lambda match: self.replacements[match.group(0)], sentence
        )


In [6]:
import pandas as pd

hatebase = pd.read_csv("outputs/hatebase.csv")
hate_words = hatebase[hatebase["language"] == "English"]["word"]
hate_words = hate_words.str.split("(").str[0].str.strip()
toxic_words = pd.concat(
    [hate_words, pd.read_csv("outputs/bad-words.csv")["word"],]
).drop_duplicates()

detoxer = SentenceDetoxer(toxic_words)

dataset_dict = dataset_dict.map(
    lambda record: {"naive-detox": detoxer(record["original"])}
)


  0%|          | 0/8679 [00:00<?, ?ex/s]

  0%|          | 0/3100 [00:00<?, ?ex/s]

  0%|          | 0/3720 [00:00<?, ?ex/s]

In [8]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox', 'naive-detox'],
        num_rows: 8679
    })
    eval: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox', 'naive-detox'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['original', 'censored', 'generated_ChrisZeng/t5-base-detox', 'generated_ChrisZeng/t5-v1_1-base-detox', 'generated_ChrisZeng/bart-base-detox', 'naive-detox'],
        num_rows: 3720
    })
})

In [ ]:
for split, dataset in dataset_dict.items():
    dataset.to_pandas().to_csv(f"outputs/generated_outputs_{split}.csv", index=False)


In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict

dataset_dict = DatasetDict(
    {
        split: Dataset.from_pandas(
            pd.read_csv(f"outputs/generated_outputs_{split}.csv")
        )
        for split in ["train", "eval", "test"]
    }
)



In [4]:
from transformers import pipeline
import pandas as pd
import numpy as np


class ToxicScore(object):
    def __init__(self, score_pipeline):
        self.score_pipeline = score_pipeline

    def __call__(self, inputs):
        return np.mean([result[0]["score"] for result in detoxify_pipeline(inputs)])


detoxify_pipeline = pipeline(
    "text-classification",
    model="unitary/toxic-bert",
    tokenizer="bert-base-uncased",
    function_to_apply="sigmoid",
    return_all_scores=True,
    device=0,
    batch_size=8,
)

toxicscore = ToxicScore(detoxify_pipeline)

from datasets import load_metric
bertscore = load_metric("bertscore")



/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [6]:
from datasets import load_metric
import numpy as np
import pandas as pd
from IPython.display import display, Pretty

rouge = load_metric("rouge")
exact_match = load_metric("exact_match")
bertscore = load_metric("bertscore")


def compute_metrics(original, generated):
    return {
        "similarity": np.mean(
            bertscore.compute(predictions=generated, references=original, lang="en")[
                "f1"
            ]
        ),
        "toxicity": np.mean(toxicscore(generated)),
    }


metrics = pd.concat(
    {
        split: pd.concat(
            pd.DataFrame(
                compute_metrics(
                    dataset_dict[split]["original"], dataset_dict[split][col]
                ),
                index=[col],
            )
            for col in dataset_dict[split].column_names
        )
        for split in dataset_dict.keys()
    },
    names=["split", "column"],
)

metrics


/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using 

similarity  toxicity
split column                                                      
train original                                  1.000005  0.634264
      censored                                  0.938401  0.119891
      generated_ChrisZeng/t5-base-detox         0.904883  0.117076
      generated_ChrisZeng/t5-v1_1-base-detox    0.909103  0.143238
      generated_ChrisZeng/bart-base-detox       0.850837  0.077703
      naive-detox                               0.978006  0.396886
eval  original                                  1.000005  0.638489
      censored                                  0.939462  0.120043
      generated_ChrisZeng/t5-base-detox         0.906586  0.120961
      generated_ChrisZeng/t5-v1_1-base-detox    0.910119  0.143255
      generated_ChrisZeng/bart-base-detox       0.852197  0.081079
      naive-detox                               0.978564  0.391680
test  original                                  1.000005  0.629271
      censored                                  0.939285  0.111152
      generated_ChrisZeng/t5-base-detox         0.906484  0.112214
      generated_ChrisZeng/t5-v1_1-base-detox    0.909312  0.133200
      generated_ChrisZeng/bart-base-detox       0.855023  0.081856
      naive-detox                               0.978402  0.386676

In [2]:
from datasets import load_metric
import numpy as np
import pandas as pd
from IPython.display import display, Pretty

rouge = load_metric("rouge")
exact_match = load_metric("exact_match")
bertscore = load_metric("bertscore")


def compute_metrics(predictions, targets):
    return {
        **{
            key: value.mid.fmeasure
            for key, value in rouge.compute(
                predictions=predictions, references=targets
            ).items()
        },
        "exact_match_rate": 0.01
        * exact_match.compute(predictions=predictions, references=targets)[
            "exact_match"
        ],
        "mean_bertscore_f1": np.mean(
            bertscore.compute(predictions=predictions, references=targets, lang="en")[
                "f1"
            ]
        ),
    }


metrics = pd.concat(
    {
        split: pd.concat(
            pd.DataFrame(
                compute_metrics(
                    dataset_dict[split][col], dataset_dict[split]["censored"]
                ),
                index=[col],
            )
            for col in [
                "generated_ChrisZeng/t5-base-detox",
                "generated_ChrisZeng/t5-v1_1-base-detox",
                "generated_ChrisZeng/bart-base-detox",
                "naive-detox",
            ]
        )
        for split in dataset_dict.keys()
    },
    names=["split", "column"],
)

metrics


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
/home/chris-zeng/csci544-project/apex-env/lib/python3.10/site-packages/torch/nn/modules/module.py:1402: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


rouge1    rouge2    rougeL  \
split column                                                                 
train generated_ChrisZeng/t5-base-detox       0.600045  0.527522  0.596211   
      generated_ChrisZeng/t5-v1_1-base-detox  0.573912  0.498893  0.569687   
      generated_ChrisZeng/bart-base-detox     0.327116  0.160506  0.323587   
      naive-detox                             0.813183  0.742482  0.808027   
eval  generated_ChrisZeng/t5-base-detox       0.603885  0.523222  0.599584   
      generated_ChrisZeng/t5-v1_1-base-detox  0.584622  0.506323  0.580477   
      generated_ChrisZeng/bart-base-detox     0.332237  0.166057  0.329100   
      naive-detox                             0.818039  0.747882  0.812162   
test  generated_ChrisZeng/t5-base-detox       0.593605  0.514198  0.588873   
      generated_ChrisZeng/t5-v1_1-base-detox  0.574319  0.496103  0.570078   
      generated_ChrisZeng/bart-base-detox     0.339565  0.176524  0.335743   
      naive-detox                             0.814202  0.745184  0.809009   

                                              rougeLsum  exact_match_rate  \
split column                                                                
train generated_ChrisZeng/t5-base-detox        0.596894          0.028920   
      generated_ChrisZeng/t5-v1_1-base-detox   0.570358          0.091600   
      generated_ChrisZeng/bart-base-detox      0.324265          0.051734   
      naive-detox                              0.809031          0.103583   
eval  generated_ChrisZeng/t5-base-detox        0.599861          0.020323   
      generated_ChrisZeng/t5-v1_1-base-detox   0.580760          0.080323   
      generated_ChrisZeng/bart-base-detox      0.329713          0.043871   
      naive-detox                              0.813399          0.109355   
test  generated_ChrisZeng/t5-base-detox        0.589002          0.016129   
      generated_ChrisZeng/t5-v1_1-base-detox   0.570247          0.079839   
      generated_ChrisZeng/bart-base-detox      0.336342          0.042473   
      naive-detox                              0.809494          0.112903   

                                              mean_bertscore_f1  
split column                                                     
train generated_ChrisZeng/t5-base-detox                0.915087  
      generated_ChrisZeng/t5-v1_1-base-detox           0.912732  
      generated_ChrisZeng/bart-base-detox              0.882213  
      naive-detox                                      0.948864  
eval  generated_ChrisZeng/t5-base-detox                0.914306  
      generated_ChrisZeng/t5-v1_1-base-detox           0.913912  
      generated_ChrisZeng/bart-base-detox              0.882438  
      naive-detox                                      0.950080  
test  generated_ChrisZeng/t5-base-detox                0.913071  
      generated_ChrisZeng/t5-v1_1-base-detox           0.912747  
      generated_ChrisZeng/bart-base-detox              0.883253  
      naive-detox                                      0.949808